## Cabeçalho
### Objetivo
Limpar/Tratar os dados de votacoes disponíveis em tb_votacoes_bronze extraídos da API de dados abertos da Câmara dos Depoutados.

**Sobre
Data de crição**: 30/09/2025

**Responsável:** Wandrieli Nery Barbosa

Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Transformação

In [0]:
%sql
select * from tiller.bronze.tb_votacoes_bronze;

In [0]:
set_catalog_and_schema_silver() #Define catalog e schema conforme run

df_votacoes_bronze = spark.table("tiller.bronze.tb_votacoes_bronze") # Carrega a tabela bronze

# Transformações
df_votacoes_silver = (
    df_votacoes_bronze
    .withColumn("data", F.to_date("data", "yyyy-MM-dd"))
    .withColumn("dataHoraRegistro", F.to_timestamp("dataHoraRegistro"))
    .withColumn("descricao", F.initcap(F.trim("descricao")))
    .withColumn(
        "resultado_votacao",
        F.when(F.col("aprovacao") == 1, F.lit("Aprovada"))
         .when(F.col("aprovacao") == 0, F.lit("Rejeitada"))
         .otherwise(F.lit("Desconhecida")))
    .withColumn("siglaOrgao", F.upper(F.trim("siglaOrgao")))
    .withColumn("data_ingestao_silver", F.current_timestamp()) # adiciona coluna de controle da camada silver
)

# Salva a tabela em Delta 
df_votacoes_silver.write.format("delta").mode("overwrite").saveAsTable("tb_votacoes_silver")

print("Tabela 'tb_votacoes_silver' criada/atualizada com sucesso!")


In [0]:
%sql
select * from tiller.silver.tb_votacoes_silver;